In [0]:
#               CS6350 BIG DATA MANAGEMENT & ANALYTICS
################################################################
# Assignement 3
# Students:
# Abirami Dhayalan  - axd210002
# Harish Srinivasan - hxs200044
#################################################################
 
# Question 2

In [0]:
from graphframes import GraphFrame
from pyspark.sql.functions import desc

In [0]:
def social_network_analysis(ip_path, op_path):
    reddit = spark.read.option("header","true").option("sep", "\t").csv(ip_path)
    
    redditVertices = reddit.select("SOURCE_SUBREDDIT").toDF("id").distinct()
    redditEdges = reddit.withColumnRenamed("SOURCE_SUBREDDIT", "src").withColumnRenamed("TARGET_SUBREDDIT", "dst")
    SubredditGraph = GraphFrame(redditVertices, redditEdges)
    SubredditGraph.cache()
    
    print( "Total Number of Subreddits: " + str(SubredditGraph.vertices.count()))
    print( "Total Number of Links in Graph: " + str(SubredditGraph.edges.count()))
    print( "Total Number of Links in Original Data: " + str(reddit.count()))
    
    SubredditGraph.edges.groupBy("src", "dst").count().orderBy(desc("count")).show(10)
    
    outDeg = SubredditGraph.outDegrees
    outDeg.orderBy(desc("outDegree")).show(5, False)
    outDeg.coalesce(1).write.option("header", "true").csv(op_path + '0')
    
    inDeg = SubredditGraph.inDegrees
    inDeg.orderBy(desc("inDegree")).show(5, False)
    inDeg.coalesce(1).write.option("header", "true").csv(op_path + '1')
    
    ranks = SubredditGraph.pageRank(resetProbability=0.15, maxIter=5)
    ranks.vertices.orderBy(desc("pagerank")).select("id", "pagerank").show(5)
    ranks.vertices.orderBy(desc("pagerank")).select("id", "pagerank").coalesce(1).write.option("header", "true").csv(op_path + '2')
    
    sc.setCheckpointDir('/Users/hxs200044@utdallas.edu/graphframes_cps')
    
    cc = SubredditGraph.connectedComponents()
    cc.groupBy("component").count().orderBy(desc("count")).show(5)
    cc.groupBy("component").count().orderBy(desc("count")).coalesce(1).write.option("header", "true").csv(op_path + '3')
    
    TriCount = SubredditGraph.triangleCount()
    TriCount.orderBy(desc("count")).show(5);
    TriCount.orderBy(desc("count")).coalesce(1).write.option("header", "true").csv(op_path + '4')
    
    #df.write.format("csv").mode("overwrite").save("/my-output/default-csv")

In [0]:
social_network_analysis("/FileStore/tables/soc_redditHyperlinks_body.tsv", "/FileStore/Reddit_Social_Network_Analysis")

Total Number of Subreddits: 27863
Total Number of Links in Graph: 286561
Total Number of Links in Original Data: 286561
+------------------+---------------+-----+
|               src|            dst|count|
+------------------+---------------+-----+
|trendingsubreddits|      changelog|  548|
|     streetfighter|            sf4|  279|
|          buildapc|  buildapcforme|  244|
|   mushroomkingdom|       gamesale|  243|
|       bluejackets| bluejacketsgwg|  223|
|    summonerschool|leagueoflegends|  220|
|          atletico|         soccer|  211|
|    writingprompts|      askreddit|  208|
|  fireemblemcasual|     fireemblem|  206|
|         babymetal|   sakuragakuin|  188|
+------------------+---------------+-----+
only showing top 10 rows

+---------------+---------+
|id             |outDegree|
+---------------+---------+
|subredditdrama |4665     |
|circlebroke    |2358     |
|shitliberalssay|1968     |
|outoftheloop   |1958     |
|copypasta      |1824     |
+---------------+---------+
